In [ ]:
%load_ext autoreload
%autoreload 2

%cd '..'

In [ ]:
import numpy as np
import pandas as pd

from sentence_transformers import SentenceTransformer

from preprocessing.utils import (
    load_event_comments,
    save_event_comments,
)

from load.utils import load_comments, save_df_as_parquet


In [ ]:
import logging
import sys

logging.basicConfig(
    level=logging.INFO,
    format="%(asctime)s [%(levelname)s] %(message)s",
    handlers=[
        logging.FileHandler("data/logs/comment_embeddings.log"),
        logging.StreamHandler(stream=sys.stdout)
    ]
)

In [ ]:
model = SentenceTransformer("all-MiniLM-L6-v2")


In [ ]:
EVENT_KEY = "mass_shootings"

logging.info(f"Loading comments of event {EVENT_KEY}")

event_comments = load_event_comments(EVENT_KEY).sample(100)


In [ ]:
logging.info(f"Computing embeddings for {len(event_comments)} comments...")

embeddings = model.encode(
    event_comments["body_cleaned"].values,  # type: ignore
    show_progress_bar=True,
    normalize_embeddings=True,
    convert_to_numpy=False,
)


In [ ]:
event_comments["embedding"] = embeddings

event_comments["embedding"] = event_comments["embedding"].apply(np.array)  # type: ignore


In [ ]:
logging.info(f"Saving embeddings for comments...")

save_event_comments(event_comments, f"{EVENT_KEY}_with_embeddings")


In [ ]:
logging.info("Computing user embeddings...")

# group by user and aggregate using np mean (axis=0)
user_embeddings = event_comments.groupby(by="author")["embedding"].apply(
    lambda x: pd.Series(
        {
            "mean": np.mean(x, axis=0),
            "max": np.maximum.reduce(x.tolist(), axis=0),
        }
    )
).to_frame()


In [ ]:
logging.info(f"Saving {len(user_embeddings)} user embeddings...")

save_df_as_parquet(
    data=user_embeddings,
    target_file=f"{EVENT_KEY}_user_embeddings.parquet",
)
